### Implementing Data Validation using Protobuf in a Distributed System
**Description**: Use gRPC to implement a distributed system that validates messages using
Protobuf.

**Steps**:
1. Create a .proto file for gRPC service.
2. Implement server-side validation
    - Create a gRPC server
    - Bind the server to an address
    - Start server

In [1]:
!pip install grpcio grpcio-tools


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 13.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 35.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Write your code from here
!python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. validator.proto


In [3]:
from concurrent import futures
import grpc
import re

import validator_pb2
import validator_pb2_grpc

class ValidatorService(validator_pb2_grpc.ValidatorServiceServicer):
    def Validate(self, request, context):
        # Simple validation logic
        if not request.username.strip():
            return validator_pb2.ValidationResult(is_valid=False, message="Username is required.")
        if request.age < 0 or request.age > 120:
            return validator_pb2.ValidationResult(is_valid=False, message="Invalid age.")
        if not re.match(r"[^@]+@[^@]+\.[^@]+", request.email):
            return validator_pb2.ValidationResult(is_valid=False, message="Invalid email format.")

        return validator_pb2.ValidationResult(is_valid=True, message="Valid user.")

def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    validator_pb2_grpc.add_ValidatorServiceServicer_to_server(ValidatorService(), server)
    server.add_insecure_port('[::]:50051')
    print("✅ gRPC server running on port 50051...")
    server.start()
    server.wait_for_termination()

if __name__ == "__main__":
    serve()


✅ gRPC server running on port 50051...


KeyboardInterrupt: 